In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import random
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pywt import wavedec

2023-10-14 21:33:08.208037: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-14 21:33:08.367883: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-14 21:33:08.367910: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-14 21:33:08.368777: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-14 21:33:08.443439: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-14 21:33:08.444265: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
import os
dataset_path = '../Preprocessing/test'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          file_path = os.path.join(dirpath2, f)
          print("Processing file: ", file_path)
          df = pd.read_csv(file_path)

          X = df.drop(['label', 'filename'], axis=1)
          y = df.iloc[:, -1]

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          lb = LabelEncoder()
          X_train = np.expand_dims(X_train, axis=2)
          X_test = np.expand_dims(X_test, axis=2)
          y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
          y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

          model = tf.keras.Sequential([
              # tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=True),
              # tf.keras.layers.Dense(32,activation='relu'),

              # tf.keras.layers.Flatten(),
              # tf.keras.layers.Dense(64, activation='relu'),
              # tf.keras.layers.Dense(32, activation='relu'),

              # tf.keras.layers.BatchNormalization(),
              
              # tf.keras.layers.Dense(5, activation='softmax')
              tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=True),
              tf.keras.layers.Dense(32,activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),

              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(64, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              tf.keras.layers.Dense(32, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),

              tf.keras.layers.BatchNormalization(),

              tf.keras.layers.Dense(5, activation='softmax')
          ])

          optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
          model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])

          history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))
          # history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

          test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

          to_append = f'{file_path}'
          to_append += f' {test_accuracy}'
          file = open('../Preprocessing/hasil.csv', 'a', newline='')
          with file:
              writer = csv.writer(file)
              writer.writerow(to_append.split())

Processing file:  ../Preprocessing/test/db1/data_MfccDwtChromaCqtdb1L9.csv
Epoch 1/50
45/45 [==============================] - 4s 34ms/step - loss: 1.5130 - acc: 0.3514 - val_loss: 1.5727 - val_acc: 0.3944
Epoch 2/50
45/45 [==============================] - 1s 22ms/step - loss: 1.1437 - acc: 0.5681 - val_loss: 1.5367 - val_acc: 0.5333
Epoch 3/50
45/45 [==============================] - 1s 23ms/step - loss: 0.9966 - acc: 0.6722 - val_loss: 1.4868 - val_acc: 0.5778
Epoch 4/50
45/45 [==============================] - 1s 22ms/step - loss: 0.8734 - acc: 0.7083 - val_loss: 1.4405 - val_acc: 0.5722
Epoch 5/50
45/45 [==============================] - 1s 23ms/step - loss: 0.8150 - acc: 0.7417 - val_loss: 1.3861 - val_acc: 0.6056
Epoch 6/50
45/45 [==============================] - 1s 22ms/step - loss: 0.7520 - acc: 0.7819 - val_loss: 1.3255 - val_acc: 0.5889
Epoch 7/50
45/45 [==============================] - 1s 22ms/step - loss: 0.6945 - acc: 0.7736 - val_loss: 1.2436 - val_acc: 0.6444
Epoch 8/